In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
list = glob.glob('./1027/*')
list.sort()

In [ ]:
realCIR = np.empty((0,64))
imagCIR = np.empty((0,64))
GT = np.empty(0)

In [ ]:
for i in range(len(list)):
    data = pd.read_excel(list[i])
    data = np.array(data.iloc[:64,:]).T
    realCIR = np.vstack((realCIR, data[0::2, :].astype(dtype=np.float32)))
    imagCIR = np.vstack((imagCIR, data[1::2, :].astype(dtype=np.float32)))
    GT = np.append(GT, np.ones(500) * i)
    
magCIR = np.sqrt(realCIR ** 2 + imagCIR ** 2)

In [3]:
realCIR = np.load('real.npy')
imagCIR = np.load('imag.npy')
magCIR = np.load('dataset.npy')
GT = np.load('label.npy')

In [4]:
print(realCIR.shape, imagCIR.shape, magCIR.shape, GT.shape)

(33000, 64) (33000, 64) (33000, 64) (33000,)


In [5]:
tan = np.arctan2(imagCIR, realCIR)

In [6]:
tan = np.arctan2(imagCIR, realCIR)
diff = tan[:,:63] - tan[:, 1:]

In [7]:
(magCIR.shape[0],magCIR.shape[1])

(33000, 64)

In [8]:
diffrence = np.zeros((magCIR.shape[0],magCIR.shape[1]))

In [9]:
diffrence.shape

(33000, 64)

In [10]:
diffrence[:,1:] = diff

In [11]:
# array = np.stack([magCIR,diffrence],axis=-1)
array = np.stack([magCIR,tan],axis=-1)

In [12]:
array.shape

(33000, 64, 2)

In [13]:
GT.shape

(33000,)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(array, GT, test_size=0.2, shuffle=True, stratify=GT)

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(128,3, activation=tf.nn.relu,input_shape=(64,2),padding='same'),
    tf.keras.layers.LayerNormalization(axis=-1),
    tf.keras.layers.Conv1D(256,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.LayerNormalization(axis=-1),
    tf.keras.layers.MaxPooling1D(2,padding='same'),
    tf.keras.layers.LayerNormalization(axis=-1),
    tf.keras.layers.Conv1D(512,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.LayerNormalization(axis=-1),
    tf.keras.layers.MaxPooling1D(2,padding='same'),
    tf.keras.layers.LayerNormalization(axis=-1),
    tf.keras.layers.Conv1D(1024,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.LayerNormalization(axis=-1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

2023-03-06 12:29:35.041575: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
X_train.shape

(26400, 64, 2)

In [18]:
input_shape = (2400,64,2) # [batch, timestamp, channel]
x=tf.random.normal(input_shape)

In [19]:
x.shape

TensorShape([2400, 64, 2])

In [20]:
y=tf.keras.layers.Flatten(input_shape=(64,2))(x)
y.shape

TensorShape([2400, 128])

In [24]:
x[0].shape

TensorShape([64, 2])

In [26]:
x[0]

<tf.Tensor: shape=(64, 2), dtype=float32, numpy=
array([[ 0.7597858 ,  1.1772699 ],
       [-1.1490649 , -0.7326539 ],
       [ 0.65414137,  0.27801362],
       [-1.2431781 , -0.47762942],
       [-0.14788699,  0.10131846],
       [ 0.3727815 ,  1.0086399 ],
       [-0.9959309 ,  0.6754406 ],
       [-0.60709476, -0.08735424],
       [-1.5807495 , -0.86369175],
       [ 0.96999305, -0.7728098 ],
       [-0.6167098 ,  1.0034144 ],
       [-1.1028564 ,  0.3568835 ],
       [ 1.3248631 , -0.5881502 ],
       [-1.8175312 ,  0.6240082 ],
       [ 0.05119636,  2.8351545 ],
       [ 0.2912606 , -1.1835668 ],
       [-0.49003282,  0.1268585 ],
       [-0.8115728 ,  1.6262712 ],
       [-0.7347959 , -0.44444007],
       [-0.18924038,  0.51921684],
       [ 0.5059274 ,  0.7412656 ],
       [-1.1626931 ,  0.4279719 ],
       [ 0.00661233, -0.84810525],
       [ 0.6556558 ,  0.06868128],
       [-0.03553225,  0.62374353],
       [-1.4947113 , -1.0859425 ],
       [-1.6110007 , -0.86005694],
      

In [25]:
y[0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.7597858 ,  1.1772699 , -1.1490649 , -0.7326539 ,  0.65414137,
        0.27801362, -1.2431781 , -0.47762942, -0.14788699,  0.10131846,
        0.3727815 ,  1.0086399 , -0.9959309 ,  0.6754406 , -0.60709476,
       -0.08735424, -1.5807495 , -0.86369175,  0.96999305, -0.7728098 ,
       -0.6167098 ,  1.0034144 , -1.1028564 ,  0.3568835 ,  1.3248631 ,
       -0.5881502 , -1.8175312 ,  0.6240082 ,  0.05119636,  2.8351545 ,
        0.2912606 , -1.1835668 , -0.49003282,  0.1268585 , -0.8115728 ,
        1.6262712 , -0.7347959 , -0.44444007, -0.18924038,  0.51921684,
        0.5059274 ,  0.7412656 , -1.1626931 ,  0.4279719 ,  0.00661233,
       -0.84810525,  0.6556558 ,  0.06868128, -0.03553225,  0.62374353,
       -1.4947113 , -1.0859425 , -1.6110007 , -0.86005694,  1.9756137 ,
       -0.0843918 , -0.36472166, -0.65521556,  0.41295004,  0.03752564,
        1.444975  ,  1.3024379 ,  1.9704639 ,  0.729888  , -1.7008336 ,
        0.704705

In [12]:
# y=tf.keras.layers.Conv1D(128,2,activation='relu',input_shape=(63,2))(x)
y=tf.keras.layers.Conv1D(6,5,activation='relu',input_shape=(64,2))(x)
y.shape

TensorShape([2400, 60, 6])

In [13]:
y1 = tf.keras.layers.AveragePooling1D(2,padding='same')(y)
y1.shape

TensorShape([2400, 30, 6])

In [ ]:
y2=tf.keras.layers.Conv1D(16, 5,activation='relu',padding='same')(y1)
y2.shape

In [ ]:
y3=tf.keras.layers.AveragePooling1D(2)(y2)
y3.shape

In [ ]:
y4=tf.keras.layers.Conv1D(120,5,activation='relu',padding='same')(y3)
y4.shape

In [ ]:
y5=tf.keras.layers.Flatten()(y4)
y5.shape

In [ ]:
y6=tf.keras.layers.Conv1D(32, 3,activation='relu',padding='same')(y5)
y6.shape

In [ ]:
y7=tf.keras.layers.MaxPooling1D(2)(y6)
y7.shape

In [ ]:
y8=tf.keras.layers.Flatten()(y7)
y8.shape

In [9]:
def step_decay(epoch):
    start = 0.0001
    drop = 0.1
    epochs_drop = 5
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

In [10]:
lr_scheduler = LearningRateScheduler(step_decay, verbose=1)

In [11]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train, epochs=10,
            validation_data=(X_test, y_test),
             callbacks=[lr_scheduler])


Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/10
450/450 [==============================] - 108s 240ms/step - loss: 1.1149 - accuracy: 0.5678 - val_loss: 0.6699 - val_accuracy: 0.7539 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0001.
Epoch 2/10
450/450 [==============================] - 109s 242ms/step - loss: 0.5527 - accuracy: 0.7923 - val_loss: 0.4885 - val_accuracy: 0.8136 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0001.
Epoch 3/10
450/450 [==============================] - 110s 244ms/step - loss: 0.3638 - accuracy: 0.8656 - val_loss: 0.4295 - val_accuracy: 0.8417 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0001.
Epoch 4/10
450/450 [==============================] - 111s 246ms/step - loss: 0.2410 - accuracy: 0.9136 - val_loss: 0.4162 - val_accuracy: 0.8536 - lr: 1.0000e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0001.
Epoch 5/10
450/450 [===

In [13]:
model.evaluate(X_test, y_test)

113/113 [==============================] - 6s 51ms/step - loss: 0.3826 - accuracy: 0.8753


[0.382550984621048, 0.8752777576446533]